In [2]:
import os
import sys
import random
import warnings
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt

In [3]:
# pip install keras

In [4]:
from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K
import tensorflow as tf
import cv2

In [5]:
IMG_WIDTH = 640
IMG_HEIGHT = 480
IMG_CHANNELS = 3

In [6]:
train_path = '/home/vraskap3/datas/train/data/'#enter path to training data
test_path = '/home/vraskap3/datas/test/data/'#enter path to testing data
mask_path = '/home/vraskap3/datas/masks/'
output_path = '/home/vraskap3/datas/output/'

In [7]:
train_ids = next(os.walk(train_path))[1]
test_ids = next(os.walk(test_path))[1]
# print(test_ids)

In [8]:
X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)

In [9]:
print("X_train",X_train.shape)
print("Y_train",Y_train.shape)

X_train (211, 480, 640, 3)
Y_train (211, 480, 640, 1)


In [10]:
print('Getting and resizing train images and masks ... ')
sys.stdout.flush()

Getting and resizing train images and masks ... 


In [11]:
for n, ids in tqdm(enumerate(train_ids), total=len(train_ids)):
#     print("value of n" , n)
#     print("value of id" , ids)
    path = train_path + ids +'/'
#     print(path)
    image = imread(path +'frame0000.png')[:,:]
    image = resize(image, (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), mode='constant', preserve_range=True)
#     print(image)
    X_train[n] = image
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
#     print(mask)
    mask_one = imread(mask_path + ids + '.png')[:,:]
    mask_one = resize(mask_one, (IMG_HEIGHT, IMG_WIDTH, 1), mode='constant', preserve_range=True)
    mask = np.maximum(mask, mask_one)
#     print(mask)
    Y_train[n] = mask


100%|██████████| 211/211 [00:34<00:00,  6.16it/s]


In [12]:
# print(X_train[0])
# print(Y_train[0])

In [18]:
# Build U-Net model
inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='categorical_crossentropy')
# model.summary()

In [19]:
callbacks = [
    EarlyStopping(patience=5, verbose=1),
    ModelCheckpoint('model-dsbowl2018-1.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

In [ ]:
results = model.fit(X_train, Y_train, validation_split=0.1, batch_size=16, epochs=10, 
                    callbacks=callbacks)

Epoch 1/10
12/12 [==============================] - 334s 28s/step - loss: 0.0000e+00 - val_loss: 0.0000e+00

Epoch 00001: val_loss improved from inf to 0.00000, saving model to model-dsbowl2018-1.h5
Epoch 2/10
 9/12 [=====================>........] - ETA: 1:22 - loss: 0.0000e+00

In [33]:
# X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, 3), dtype=np.uint8)
# sizes_test = []
# sys.stdout.flush()
# for n,ids in tqdm(enumerate(test_ids), total=len(test_ids)):
#     path = test_path + ids +'/'
#     img = imread(path +'frame0000.png')[:,:]
#     sizes_test.append([img.shape[0], img.shape[1]])
#     img = resize(img, (IMG_HEIGHT, IMG_WIDTH, 3), mode='constant', preserve_range=True)
# #     print(ids)
#     X_test[n] = img
# #     print(type(X_test[n]))

100%|██████████| 114/114 [00:10<00:00, 10.67it/s]


In [20]:
model.load_weights('model-dsbowl2018-1.h5')

In [20]:
for x in test_ids:
    path = test_path + x +'/'
    img =imread(path +'frame0000.png')[:,:]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), mode='constant', preserve_range=True)
#     img=np.expand_dims(img,axis=-1)
    X_test = np.zeros((1, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=int)
    X_test[0] = img
    predictions = model.predict(X_test)
    mask=predictions[0]
#     print(mask)
#     print(type(mask))
    
    img_name = output_path + x + ".png"
    cv2.imwrite(img_name,mask)